In [ ]:
import matplotlib.pyplot as plt
import matplotlib.markers as mks

import plotly.graph_objs as go
import numpy as np
import pandas as pd
import pprint
import random as rn
from itertools import chain
import math
from flask import Flask

import requests

In [ ]:
NumOfVehicles = 0
Q = 0
NumOfNodes = 0
NumOfDepots = 0
NumOfCust = 0
x_Nodes = []
y_Nodes = []
x_Cust = []
y_Cust = []
xDepots = []
yDepots = []
Custs_num = []
index_of_dp_in_nodes = []
Nodes_num = []
Customers = []
Demands = []
InitialChromosoms = []
isolated_nodes = []

In [ ]:
with open("/VRP/real_data.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

for i, line in enumerate(lines, start=1):
    if i == 1:
        numbers = list(map(int, line.strip().split(",")))
        NumOfDepots = numbers[0]
        VehiclesPerDepot = numbers[1]
        NumOfCust = numbers[2]
        Q = numbers[3]
        NumOfVehicles = NumOfDepots * VehiclesPerDepot
    else:
        numbers = list(map(float, line.strip().split(",")))
        demand = numbers[1]
        x, y = numbers[3], numbers[4]
        if demand == 0:  # Depot
            index_of_dp_in_nodes.append(len(x_Nodes))
            xDepots.append(x)
            yDepots.append(y)
        else:
            x_Cust.append(x)
            y_Cust.append(y)
            Demands.append(demand)
        x_Nodes.append(x)
        y_Nodes.append(y)

NumOfNodes = NumOfCust + NumOfDepots
Nodes_num = list(range(1, NumOfNodes + 1))
GD = [['d{}'.format(k+1)] for k in range(NumOfDepots)]
Customers = list(range(NumOfDepots, NumOfNodes))

print('num_Nodes =', NumOfNodes)
print('num_Custs =', NumOfCust)
print('num_Depots =', NumOfDepots)
print('vehicles per depot =', VehiclesPerDepot)
print('Q =', Q)
print('Total vehicles =', NumOfVehicles)
print('Sum of Demands =', sum(Demands))
print('Index of depots in Nodes =', index_of_dp_in_nodes)


In [ ]:
with open("//VRP/OD_matrix.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()
Dist_Matrix = np.full((len(x_Nodes), len(y_Nodes)), np.nan)

def DistMatrix():
    l = 1
    # try:
    for i, line in enumerate(lines, start=1):
        if i == 1:
            continue
        objects = line.split(",")
        if int(objects[2]) > l:
            l += 1
            # if l == len(x_Nodes)+1:
            #     break
        Dist_Matrix[l-1][int(objects[3])-1] = float(objects[-1])
    return Dist_Matrix
    # except:
    #     print(l)

DistMatrix()
df = pd.DataFrame(Dist_Matrix)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# display(df)

In [ ]:
isolated_nodes = []
Custs_num = [i for i in range(1, NumOfNodes+1) if i not in index_of_dp_in_nodes]
for i in range(len(Dist_Matrix)):
    if i == 0 and np.all(np.isnan(Dist_Matrix[i, i+1:])):
            isolated_nodes.append(i)
    elif i == len(Dist_Matrix)-1 and np.all(np.isnan(Dist_Matrix[0, :i])):
            isolated_nodes.append(i)
    if np.all(np.isnan(Dist_Matrix[i, :i])) and np.all(np.isnan(Dist_Matrix[i, i+1:])):
        isolated_nodes.append(i+1) 
        # Custs_num.remove(i)
print("isolated_nodes = ", isolated_nodes)
# print("Custs_num = ", Custs_num)

In [ ]:
def Grouping(num_depots, dist_matrix, custs_num, depot_indices, isolated):
    groups = [['d{}'.format(i + 1)] for i in range(num_depots)]

    for i in custs_num:
        if i in isolated:
            continue  
        dist_to_depots = []
        for j in depot_indices:
            d = dist_matrix[i - 1][j]
            dist_to_depots.append(d)
        if all(np.isnan(dist_to_depots)):
            continue  
        min_index = np.nanargmin(dist_to_depots)  
        groups[min_index].append(i)

    return groups


In [ ]:
GD = Grouping(NumOfDepots, Dist_Matrix, Custs_num, index_of_dp_in_nodes, isolated_nodes)
print("Grouping Result:", GD)


In [ ]:
import random as rn

def Initialpopulation(population_num: int, GD, isolated_nodes):
    population = []

    for _ in range(population_num):
        chromosome = {}
        for group in GD:
            depot_label = group[0]  # e.g., 'd1'
            customers = [c for c in group[1:] if c not in isolated_nodes]
            rn.shuffle(customers)  # inplace shuffle
            chromosome[depot_label] = customers
        population.append(chromosome)

    return population


InitialChromosoms = Initialpopulation(50, GD, isolated_nodes)
print("Sample Chromosome:\n", InitialChromosoms[0])


In [ ]:
chromosomdist = []
def Sumdist(path):
    total = 0
    for i in range(len(path) - 1):
        total += Dist_Matrix[path[i] - 1][path[i + 1] - 1]
    return total


# demands_dict = {Custs_num[i]: Demands[i] for i in range(len(Custs_num))}
def SumDemands(Route):
    sumdemands = 0
    for i in Route:
        sumdemands = sumdemands + Demands[Custs_num.index(i)]
    return sumdemands

In [ ]:
def fitness_function(chromosome, dist_matrix, depot_indices, demands_dict, Q, vehicles_per_depot, penalty_coef=10000):
    total_cost = 0
    infeasible_penalty = 0

    for depot_id, route in chromosome.items():
        depot_index = depot_indices[int(depot_id[-1]) - 1]  # 'd1' -> 0, 'd2' -> 1
        vehicle_count = 0
        current_demand = 0
        current_path = [depot_index]

        for customer in route:
            demand = demands_dict.get(customer, 0)

            if current_demand + demand <= Q:
                current_path.append(customer)
                current_demand += demand
            else:
                
                current_path.append(depot_index)
                total_cost += Sumdist(current_path)

                vehicle_count += 1
                current_path = [depot_index, customer]
                current_demand = demand

        
        current_path.append(depot_index)
        total_cost += Sumdist(current_path)
        vehicle_count += 1

        
        if vehicle_count > vehicles_per_depot:
            infeasible_penalty += (vehicle_count - vehicles_per_depot) * penalty_coef

    return total_cost + infeasible_penalty


In [ ]:
fitness_scores = []
for chrom in InitialChromosoms:
    score = fitness_function(
        chrom,
        Dist_Matrix,
        index_of_dp_in_nodes,
        demands_dict,
        Q,
        VehiclesPerDepot,
        penalty_coef=10000
    )
    fitness_scores.append(score)

print("Fitness scores for population:", fitness_scores[:3])


In [ ]:
import random

def tournament_selection(population, fitness_scores, num_parents, k=3):
    selected_parents = []

    for _ in range(num_parents):
        
        indices = random.sample(range(len(population)), k)
        tournament = [(i, fitness_scores[i]) for i in indices]
       
        winner_index = min(tournament, key=lambda x: x[1])[0]
        selected_parents.append(population[winner_index])

    return selected_parents


In [ ]:

parents = tournament_selection(InitialChromosoms, fitness_scores, num_parents=20, k=3)

print(f"Selected {len(parents)} parents.")
print("Sample parent fitness:", fitness_function(parents[0], Dist_Matrix, index_of_dp_in_nodes, demands_dict, Q, VehiclesPerDepot))


In [ ]:
def ordered_crossover(parent1, parent2):
    size = len(parent1)
    a, b = sorted(random.sample(range(size), 2))
    child = [None] * size

    
    child[a:b+1] = parent1[a:b+1]

    
    fill_pos = (b + 1) % size
    for i in range(size):
        p2_gene = parent2[(b + 1 + i) % size]
        if p2_gene not in child:
            child[fill_pos] = p2_gene
            fill_pos = (fill_pos + 1) % size

    return child


In [ ]:
def crossover_parents(parent1, parent2):
    child = {}
    for depot in parent1.keys():
        route1 = parent1[depot]
        route2 = parent2[depot]
        child[depot] = ordered_crossover(route1, route2)
    return child


In [ ]:
def generate_offspring(parents, num_offspring):
    offspring = []
    while len(offspring) < num_offspring:
        p1, p2 = random.sample(parents, 2)
        child = crossover_parents(p1, p2)
        offspring.append(child)
    return offspring


In [ ]:
offspring = generate_offspring(parents, num_offspring=30)
print(f"Generated {len(offspring)} offspring.")
print("Sample offspring fitness:", fitness_function(offspring[0], Dist_Matrix, index_of_dp_in_nodes, demands_dict, Q, VehiclesPerDepot))


In [ ]:
def mutate_chromosome(chromosome, mutation_rate=0.2):
    new_chrom = {}

    for depot, route in chromosome.items():
        route = route.copy()
        if random.random() < mutation_rate:
            i, j = sorted(random.sample(range(len(route)), 2))
            route[i], route[j] = route[j], route[i]
        new_chrom[depot] = route

    return new_chrom


In [ ]:
mutated_offspring = mutate_population(offspring, mutation_rate=0.3)
print("Mutation applied to offspring.")
print("Mutated sample fitness:", fitness_function(mutated_offspring[0], Dist_Matrix, index_of_dp_in_nodes, demands_dict, Q, VehiclesPerDepot))


In [ ]:
def run_genetic_algorithm(
    initial_population,
    dist_matrix,
    depot_indices,
    demands_dict,
    Q,
    vehicles_per_depot,
    num_generations=100,
    num_parents=20,
    num_offspring=30,
    mutation_rate=0.2,
    elitism_size=2,
    penalty_coef=10000,
    verbose=True
):
    population = initial_population.copy()
    best_fitness_per_gen = []

    for generation in range(num_generations):
        
        fitness_scores = [
            fitness_function(ch, dist_matrix, depot_indices, demands_dict, Q, vehicles_per_depot, penalty_coef)
            for ch in population
        ]

        
        best_fitness = min(fitness_scores)
        best_fitness_per_gen.append(best_fitness)
        if verbose:
            print(f"Generation {generation+1}: Best fitness = {best_fitness:,.2f}")

       
        parents = tournament_selection(population, fitness_scores, num_parents=num_parents)

       
        offspring = generate_offspring(parents, num_offspring=num_offspring)

       
        mutated_offspring = mutate_population(offspring, mutation_rate=mutation_rate)

        
        elite_indices = sorted(range(len(fitness_scores)), key=lambda i: fitness_scores[i])[:elitism_size]
        elites = [population[i] for i in elite_indices]

        
        population = elites + mutated_offspring

    
    final_fitness_scores = [
        fitness_function(ch, dist_matrix, depot_indices, demands_dict, Q, vehicles_per_depot, penalty_coef)
        for ch in population
    ]
    best_index = final_fitness_scores.index(min(final_fitness_scores))
    best_solution = population[best_index]

    return best_solution, best_fitness_per_gen


In [ ]:
best_solution, fitness_curve = run_genetic_algorithm(
    initial_population=InitialChromosoms,
    dist_matrix=Dist_Matrix,
    depot_indices=index_of_dp_in_nodes,
    demands_dict=demands_dict,
    Q=Q,
    vehicles_per_depot=VehiclesPerDepot,
    num_generations=1000,
    num_parents=20,
    num_offspring=30,
    mutation_rate=0.2,
    elitism_size=2,
    penalty_coef=10000,
    verbose=True
)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def plot_routes(best_solution, depot_indices, x_Nodes, y_Nodes, title="Optimized Routes"):
    depot_colors = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#a65628']
    plt.figure(figsize=(14, 10))
    legend_patches = []

    for idx, (depot_label, customer_list) in enumerate(best_solution.items()):
        depot_index = depot_indices[int(depot_label[-1]) - 1]
        depot_x = x_Nodes[depot_index]
        depot_y = y_Nodes[depot_index]
        color = depot_colors[idx % len(depot_colors)]

        
        path = [depot_index]
        current_demand = 0
        for cust in customer_list:
            demand = demands_dict.get(cust, 0)
            if current_demand + demand <= Q:
                path.append(cust - 1)
                current_demand += demand
            else:
                draw_path(path, x_Nodes, y_Nodes, color)
                path = [depot_index, cust - 1]
                current_demand = demand
        draw_path(path, x_Nodes, y_Nodes, color)

        
        plt.scatter(
            depot_x,
            depot_y,
            marker='*',  
            s=400,
            color=color,
            edgecolors='black',
            linewidths=1.5,
            zorder=6,
            label=depot_label
        )

        plt.text(
            depot_x,
            depot_y - 1.5,  
            depot_label,
            fontsize=12,
            fontweight='bold',
            color=color,
            ha='center',
            bbox=dict(facecolor='white', edgecolor=color, boxstyle='round,pad=0.3')
        )

   
        legend_patches.append(
            mpatches.Patch(color=color, label=f"{depot_label} Routes")
        )

    
    plt.title(title, fontsize=18, fontweight='bold')
    plt.xlabel("X Coordinate", fontsize=12)
    plt.ylabel("Y Coordinate", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.legend(handles=legend_patches, fontsize=12)
    plt.tight_layout()
    plt.show()


In [ ]:
def draw_path(path, x_Nodes, y_Nodes, color):
    x_coords = [x_Nodes[i] for i in path]
    y_coords = [y_Nodes[i] for i in path]
    plt.plot(x_coords, y_coords, color=color, linewidth=2.5, alpha=0.8, zorder=3)
    plt.scatter(x_coords[1:], y_coords[1:], c=color, s=50, alpha=0.9, zorder=4)  

In [ ]:
import matplotlib.pyplot as plt
plt.plot(fitness_curve)
plt.xlabel("Generation")
plt.ylabel("Best Fitness")
plt.title("Fitness Over Generations")
plt.grid(True)
plt.show()
